# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
example1 = {
    "links": [
        {"type": "about page", "url": "https://aercafrica.org/about-us"},
        {"type": "careers page", "url": "https://aercafrica.org/knowledge"},
        {"type": "publications page", "url": "https://publication.aercafricalibrary.org/home"},
        {"type": "research page", "url": "https://aercafrica.org/our-research/"}
    ]
}

example2 = {
    "links": [
        {"type": "about page", "url": "https://www.who.int/about"},
        {"type": "organizational structure page", "url": "https://www.who.int/about/structure"},
        {"type": "collaboration page", "url": "https://www.who.int/about/collaboration"},
        {"type": "careers page", "url": "https://www.who.int/careers"}
    ]
}

In [8]:
multishot_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
multishot_system_prompt += f"""You should respond in JSON as in the following examples: \n
{example1} \n
{example2}
"""

In [9]:
print(multishot_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in the following examples: 

{'links': [{'type': 'about page', 'url': 'https://aercafrica.org/about-us'}, {'type': 'careers page', 'url': 'https://aercafrica.org/knowledge'}, {'type': 'publications page', 'url': 'https://publication.aercafricalibrary.org/home'}, {'type': 'research page', 'url': 'https://aercafrica.org/our-research/'}]} 

{'links': [{'type': 'about page', 'url': 'https://www.who.int/about'}, {'type': 'organizational structure page', 'url': 'https://www.who.int/about/structure'}, {'type': 'collaboration page', 'url': 'https://www.who.int/about/collaboration'}, {'type': 'careers page', 'url': 'https://www.who.int/careers'}]}



In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": multishot_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/ResembleAI/chatterbox',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/wushuang98/Direct3D-S2-v1.0-demo',
 '/spaces/NihalGazi/Text-To-Speech-Unlimited',
 '/spaces',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/yandex/yambda',
 '/datasets/MiniMaxAI/SynLogic',
 '/datasets/cognitivecomputations/china-refusals',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gramma

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'models page', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the 

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += "Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1-0528\nUpdated\n4 days ago\n•\n41.6k\n•\n1.61k\ndeepseek-ai/DeepSeek-R1-0528-Qwen3-8B\nUpdated\n4 days ago\n•\n55.8k\n•\n566\nResembleAI/chatterbox\nUpdated\n3 days ago\n•\n487\ngoogle/gemma-3n-E4B-it-litert-preview\nUpdated\n7 days ago\n•\n789\nByteDance-Seed/BAGEL-7B-MoT\nUpdated\n11 days ago\n•\n8.22k\n•\n913\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n565\n565

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

### **About Us**
Hugging Face is at the forefront of the AI community, dedicated to building the future of machine learning. Our platform serves as a collaborative space for developers, researchers, and organizations to explore, share, and innovate with machine learning models, datasets, and applications. With a commitment to open-source principles, we aim to democratize AI and empower individuals to create, discover, and collaborate more effectively.

---

### **What We Offer**

- **Models**: Access over 1 million machine learning models for various tasks, including text generation, image processing, and more.
- **Datasets**: Discover and share over 250,000 datasets, allowing users to find the perfect data for their projects.
- **Spaces**: A platform to run and share interactive applications built around machine learning and data science.
- **Enterprise Solutions**: Tailored solutions for businesses seeking advanced AI capabilities with enterprise-grade security and dedicated support.

---

### **Our Community**

Join a vibrant and thriving community of over 50,000 organizations and contributors. Leaders in AI, including giants like Google, Microsoft, and Amazon, leverage our platform to advance their projects. Collaborate, learn, and grow within a network driven by innovation and open exchange.

---

### **Our Culture**

At Hugging Face, we pride ourselves on fostering a collaborative and inclusive culture. Our mission is complemented by a commitment to openness and transparency, enabling everyone—from enthusiasts to experts—to contribute and thrive. We celebrate diversity in our team and community, believing that varied perspectives enrich our output and creativity.

---

### **Careers at Hugging Face**

As we continue to expand and innovate, we are always on the lookout for passionate individuals who are eager to shape the future of AI. We offer a range of career opportunities across various domains, including engineering, research, community management, and more. Join us and be a part of something transformative!

- **Current Openings**: Visit our [Jobs Page](https://huggingface.co/jobs) to explore career opportunities.
- **Company Values**: Embrace openness, build with respect, and strive for inclusivity in all we do.

---

### **Join Us!**

Whether you are a prospective customer looking for cutting-edge AI solutions, an investor interested in the future of machine learning, or a recruit craving a meaningful role in a breakthrough company, Hugging Face welcomes you.

- **Explore**: Visit us at [Hugging Face](https://huggingface.co) to discover more!
- **Connect**: Follow us on social media to stay updated about the latest in AI and machine learning.

Together, let’s build the future of AI!

--- 

**Contact Us**  
For more information, inquiries, or support, reach out to us at [Contact Us](https://huggingface.co/contact).

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face Brochure

## About Us
**Hugging Face** is not just a company; we are an **AI community building the future**. Our mission is to create a collaborative platform where machine learning enthusiasts and professionals can come together to innovate and share knowledge. At Hugging Face, we believe in the power of open-source technology and diverse contributions to accelerate advancements in AI.

## What We Offer
- **Models**: Explore over **1 million models** ranging from natural language processing to computer vision.
- **Datasets**: Access and share **250,000+ datasets** tailored for various machine learning tasks.
- **Spaces**: Collaborate and demonstrate applications in our unique Spaces, featuring user-generated apps and demos.
- **Compute Solutions**: Take advantage of our robust compute offerings starting at **$0.60/hour**, perfect for deploying applications seamlessly.

## Our Community
**Trusted by over 50,000 organizations**, including major players like Google, Microsoft, Amazon, and Meta, Hugging Face serves a diverse clientele. From enterprises to non-profits, our platform has become an essential tool for AI developers aiming to push the envelope of innovation.

## Company Culture
At Hugging Face, we embrace an inclusive culture that fosters collaboration and creativity. Our team, consisting of over **217 talented professionals**, is motivated by a shared passion for AI and machine learning. We encourage a community-oriented approach where everyone’s ideas are valued and contributions are celebrated.

## Career Opportunities
Join us in our mission to democratize AI! Whether you're an engineer, researcher, or marketer, Hugging Face is always looking for talented individuals who are excited to contribute to meaningful projects. We offer competitive salaries and a flexible work environment, fostering growth and development for all employees.

## Connect With Us
Stay updated on our latest news, models, and community discussions by following us on [GitHub](https://huggingface.co), [Twitter](https://twitter.com/huggingface), and [LinkedIn](https://www.linkedin.com/company/hugging-face).

**Join the Hugging Face community today and help us build the future of AI together!**

For more information, visit our website: [huggingface.co](https://huggingface.co)

---
*Hugging Face: The AI community building the future.*


In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# 🤗 Hugging Face: The Cuddliest AI Community in the Tech Universe!

Welcome to Hugging Face, where we take the “Aww” out of AI and replace it with “Whoa!” Join us as we navigate through **models, datasets, and delightful applications**—all while hugging our way to the **future!**

### ⭐ Why Join the Hugging Face Family?

- **Models Galore**: With over **1 million models**, we’re not just collecting models, we’re building a model museum! Go ahead, swipe right on your favorite AI companion. 🕺
  
- **Dataset Delights**: Who knew datasets could be so exciting? With **250k+ datasets**, we’ve got the information you crave—whether it's for deep learning or just deep pondering. 📊

- **Spaces to Explore**: Feel like creating your own AI app? No problem! You can create, discover, and collaborate while pretending to be the next AI Picasso. 🎨

### 🎉 Our Culture: Work Hard, Hug Harder!

At Hugging Face, we believe in the power of teamwork and community. Imagine a place where everyone’s sharing ideas in a cozy circle, drinking virtual hot cocoa, and contributing to the future of AI, all while wearing fuzzy slippers. 

- **Open-Source for All**: We’re all about collaboration (and hugs of course). Our open-source framework allows everyone to contribute to the AI revolution without needing a PhD in rocket science! 🚀

- **Diversity is Key**: We attract clients from tech giants like Google, Microsoft, and Amason (yeah, we misspelled that for giggles), ensuring a vibrant and diverse workplace—because who doesn’t love a potluck of ideas (and maybe some food too)? 🥳

### 👔 Careers: Yes, We’re Hiring!

Tired of that cubicle life? Join our team where the only thing between you and success is a well-timed joke:

- **Positions Available**: We’re looking for **AI engineers, data scientists**, and **community managers** who can not only code like the wind but also crack the best puns. 

- **Flexible Work Environment**: Work from anywhere! Whether it's your comfy couch or a stylish café, wear your pajamas or fancy pants—just don't forget your hugs! 😄

### 📚 Customers: Who’s Applying the Hugs?

Over **50,000 organizations**, including heavy hitters like Meta, Amazon, and Intel, trust us to be their AI buddies. No pressure! 😅

### 🤖 Trendy Technologies

Want to build something fantastic? Use our amazing tools, like:

- **Transformers**: No, not the robot kind. But pretty darn close!
- **Text Generation Inference**: Because every good hug starts with great words.
- **Smol Agents**: Tiny agents with big dreams! 🌟

### ✨ Join Us!

Whether you’re a prospective customer, investor, or future teammate, come on down to Hugging Face! We promise to sprinkle in some fun, laughter, and, of course, lots of hugs. 

**Ready to leap into the future?** Just give us a nudge (or a hug) at [Hugging Face](https://huggingface.co).

Let’s build a brighter AI future together—because why should robots have all the fun? 🤗

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [25]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Finally, this brochure should be translated to spanish"

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussions page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## Welcome to Hugging Face
**The AI community building the future.** Hugging Face is a collaborative platform at the forefront of machine learning, where innovators unite to develop models, datasets, and applications for artificial intelligence.

### Our Offerings
- **Models:** Access over **1 million models** on our platform, allowing you to explore the latest advancements in AI.
- **Datasets:** We provide more than **250,000 datasets** to accelerate your machine learning projects.
- **Spaces:** Collaborate on innovative applications with **400,000+ apps** running in our spaces.

### Community-Centric Approach
At Hugging Face, we believe in the power of community. Over **50,000 organizations**, including leading tech giants like Google, Microsoft, and Amazon, are part of our ecosystem. Our open-source projects, such as Transformers and Diffusers, are built with input and collaboration from our users, ensuring that we create the best tools together.

### Company Culture
Hugging Face champions a culture of collaboration, diversity, and continuous learning. We foster an environment where team members can share ideas freely, work together on cutting-edge technology, and have an impact on the future of AI. Our team is passionate, forward-thinking, and dedicated to making machine learning accessible to everyone.

### Careers with Us
Join our mission to democratize AI. At Hugging Face, we are continually looking for enthusiastic and talented individuals to be part of our team. We value different perspectives and invite people from all backgrounds to contribute to our growth. Explore open positions on our [careers page](insert careers page link) and become a part of our thriving community.

### Connect with Us
Stay updated and engaged with the Hugging Face community. Follow us on:
- [GitHub](insert GitHub link)
- [Twitter](insert Twitter link)
- [LinkedIn](insert LinkedIn link)
- [Discord](insert Discord link)

---

## Hugging Face Folleto

### Bienvenido a Hugging Face
**La comunidad de IA construyendo el futuro.** Hugging Face es una plataforma colaborativa a la vanguardia del aprendizaje automático, donde los innovadores se unen para desarrollar modelos, conjuntos de datos y aplicaciones de inteligencia artificial.

### Nuestras Ofertas
- **Modelos:** Accede a más de **1 millón de modelos** en nuestra plataforma, permitiéndote explorar los últimos avances en IA.
- **Conjuntos de Datos:** Proporcionamos más de **250,000 conjuntos de datos** para acelerar tus proyectos de aprendizaje automático.
- **Espacios:** Colabora en aplicaciones innovadoras con **más de 400,000 aplicaciones** en funcionamiento en nuestros espacios.

### Enfoque Centrado en la Comunidad
En Hugging Face, creemos en el poder de la comunidad. Más de **50,000 organizaciones**, incluidas las principales empresas tecnológicas como Google, Microsoft y Amazon, forman parte de nuestro ecosistema. Nuestros proyectos de código abierto, como Transformers y Diffusers, se construyen con la contribución y colaboración de nuestros usuarios, asegurando que creemos las mejores herramientas juntos.

### Cultura de la Empresa
Hugging Face fomenta una cultura de colaboración, diversidad y aprendizaje continuo. Promovemos un entorno donde los miembros del equipo pueden compartir ideas libremente, trabajar juntos en tecnología de vanguardia y tener un impacto en el futuro de la IA. Nuestro equipo es apasionado, visionario y está dedicado a hacer que el aprendizaje automático sea accesible para todos.

### Carreras con Nosotros
Únete a nuestra misión de democratizar la IA. En Hugging Face, estamos buscando continuamente personas entusiastas y talentosas para formar parte de nuestro equipo. Valoramos diferentes perspectivas e invitamos a personas de todos los orígenes a contribuir a nuestro crecimiento. Explora las posiciones abiertas en nuestra [página de carreras](insert careers page link) y conviértete en parte de nuestra vibrante comunidad.

### Conéctate con Nosotros
Mantente informado y comprometido con la comunidad de Hugging Face. Síguenos en:
- [GitHub](insert GitHub link)
- [Twitter](insert Twitter link)
- [LinkedIn](insert LinkedIn link)
- [Discord](insert Discord link)